In [37]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict, Annotated , List
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.types import interrupt, Command
from dotenv import load_dotenv
import requests
from pathlib import Path
import operator
from pydantic import BaseModel, Field
from typing import Literal, Tuple , Union
from langchain_mistralai import ChatMistralAI
from langgraph.types import Send
import re
from langchain_community.tools import DuckDuckGoSearchRun
import os
from langchain.agents import create_agent

key = "N56JL9JhomtYzc64dQh7K05t6m0G8u9Q"

model = ChatMistralAI(
    model="mistral-large-2512",
    mistral_api_key= key,  # Direct key
    temperature=0.7
)

In [38]:
search_tool = DuckDuckGoSearchRun(region="us-en")

In [39]:
agent = create_agent(model = model)

First we have to make a share memory of workflow which is state.

In [40]:
class task(BaseModel):
    task_id : int 
    task_name : Annotated[str, Field(..., description= "Task name")]
    task_description : Annotated[str, Field(..., description= "What you have to cover in the task.")]
    

In [41]:
class Plan(BaseModel): 
    """Plan to follow in future"""
    tasks : Annotated[List[task], Field(...,description= "different tasks to follow, should be in sorted order")]

In [42]:
class Response(BaseModel):
    response : str 

In [43]:
class replan(BaseModel):
    action : Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


In [44]:
model_with_structure_replan = model.with_structured_output(replan)

In [45]:
model_with_structure_response = model.with_structured_output(Plan)

In [46]:
class State(TypedDict):
    user_input : str 
    plan : Plan
    previous_executed_steps : Annotated[List[Tuple],operator.add]
    response : str

Now we have both Planning pydantic schema and Replanning pydantic shema so now our model can give us strcuture reponse means we have control of our model response. 

In [47]:
def set_planner(State : State):
    #Now design system prompts for our both schemas 
    plan = model_with_structure_response.invoke([SystemMessage(content =f"""For the given objective, come up with a simple step by step plan. \
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
    The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps."""),
    HumanMessage(content=f"""{State["user_input"]}""")])

    return {"plan" : plan}

In [48]:
def executor(State : State):
    plan = State["plan"]
    task = plan.tasks[0]
    task_formatted = f"""For the following plan:{plan}\n\nYou are tasked with executing task {task}"""
    agent_response = agent.invoke({"messages": [("user", task_formatted)]})
    return {"previous_executed_steps": [(task, agent_response["messages"][-1].content)]}

In [49]:
def replanner(State: State):
    replan =f"""For the given objective, come up with a simple step by step plan. \
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
    The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

    Your objective was this:
    {State["user_input"]}

    Your original plan was this:
    {State["plan"]}

    You have currently done the follow steps:
    {State['previous_executed_steps']}

    Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
    plan = model_with_structure_replan.invoke(replan)
    if isinstance(plan.action ,Response):
        return {"response": plan.action.response } 
    else: 
        return {"plan" : plan.action}

In [50]:
def condition(State : State):
    if "response" in State and State["response"]:
        return END
    else: 
        return "executor"

In [51]:
graph = StateGraph(State)

#Lets make nodes 

graph.add_node("planner", set_planner)
graph.add_node("executor", executor)
graph.add_node("replanner",replanner)

graph.add_edge(START, "planner")
graph.add_edge("planner", "executor")
graph.add_edge("executor","replanner")
graph.add_conditional_edges("replanner", condition ,["executor",END])

workflow = graph.compile()

In [52]:
result = workflow.invoke({"user_input":"Plan a trip to newyork city 5000$ budget book hotel book cab book ride plan it for 7 days and also plan breakfast dinner and vsit place for each day"})

In [53]:
result 

{'user_input': 'Plan a trip to newyork city 5000$ budget book hotel book cab book ride plan it for 7 days and also plan breakfast dinner and vsit place for each day',
 'plan': Plan(tasks=[task(task_id=3, task_name='Research and Book Accommodation', task_description='Search for hotels or Airbnb options in NYC within the $300/night budget. Prioritize locations in Manhattan, Brooklyn, or areas near major tourist attractions (e.g., Times Square, Central Park, or Downtown Brooklyn). Compare prices, read reviews, and check for amenities like free cancellation, Wi-Fi, and breakfast. Finalize and book accommodation for 7 nights, ensuring the total cost does not exceed $2,100.'), task(task_id=4, task_name='Plan and Book Transportation', task_description='1. **Airport Transfer**: Book a cab or rideshare (Uber/Lyft) for the arrival day from the airport to the hotel. Estimate the cost and ensure it fits within the $500 transportation budget.\n2. **Local Transportation**: Plan daily transportation 

In [ ]:
result

{'user_input': 'Plan a full fledge trip to newyork city',
 'plan': Plan(tasks=[task(task_id=2, task_name='Set a Budget', task_description='Determine the total budget for the trip to New York City, broken down into the following categories:\n  - Flights: Include round-trip airfare, baggage fees, and airport transfers.\n  - Accommodation: Calculate the total cost for the entire duration of the stay (hotel, Airbnb, or other lodging).\n  - Food: Estimate daily food expenses (breakfast, lunch, dinner, snacks) and include a buffer for fine dining or unexpected meals.\n  - Transportation: Include costs for subway passes (e.g., MetroCard), taxis, rideshares (Uber/Lyft), and any intercity travel (e.g., trains to nearby cities).\n  - Activities: List all pre-booked activities (e.g., Broadway shows, museum tickets, guided tours) and estimate costs for spontaneous activities.\n  - Miscellaneous: Allocate funds for souvenirs, tips, emergency expenses, and unexpected costs.\n  - Ensure the budget al